# Hungarian elections analysis

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
plt.rcParams["figure.figsize"] = (14, 7)

In [3]:
df_affiliation = pd.read_csv("affiliation.csv")

In [4]:
df_nominating_organization = pd.read_csv("nominating_organization.csv")

In [5]:
df_election = pd.read_csv("election.csv")

In [6]:
df_district_hierarchy = pd.read_csv("district_hierarchy.csv")

In [7]:
df_geographical_unit = pd.read_csv("geographical_unit.csv")

In [8]:
df_election_precinct = pd.read_csv("election_precinct.csv")

In [9]:
df_vote_record = pd.read_csv("vote_record.csv")

In [10]:
party_level_df = df_vote_record.merge(df_affiliation).merge(df_election_precinct)

In [11]:
# only first rounds, non individual
relevant_elections = df_election.loc[
    lambda df: ~df["is_individual"] & df["election_id"].str.endswith("1")
]

In [12]:
# parties that have received over 10% of the votes in 3 different election
parties_for_elections = (
    party_level_df.pivot_table(
        index="party", columns="election_id", values="vote_count", aggfunc="sum"
    )
    .loc[:, relevant_elections["election_id"]]
    .fillna(0)
)

In [13]:
total_votes_by_election = df_vote_record.merge(df_election_precinct).groupby("election_id")["vote_count"].sum()

In [14]:
(
    parties_for_elections.loc[
        lambda df: (df / total_votes_by_election[df.columns] > 0.03).sum(axis=1) > 2
    ]
    .rename(columns=relevant_elections.set_index("election_id")["start_date"].to_dict())
    .astype(int)
    .T.style.background_gradient(
        axis=None,
    )
    .set_caption('Number of votes gained by political parties in Hungarian elections')
)

party,DK,FIDESZ,FKGP,JOBBIK,KDNP,LMP,MDF,MSZP,PM,SZDSZ
election_id,,,,,,,,,,
2018-04-08,306853,2607899,0,1090331,2607899,399507,0,680680,680680,0
2010-04-11,0,2706282,0,855263,2706282,383173,135357,990253,0,0
2006-04-09,0,2272962,822,116463,2272962,0,271881,2336684,0,350196
1998-05-10,0,1263504,617713,0,113710,0,136180,1445794,0,350871
2014-04-06,1287384,2142112,7956,1015570,2142112,265220,0,1287384,1287384,0
1994-05-08,0,378951,476262,0,378948,0,633112,1781707,0,1065316
2022-04-03,1930740,2803778,0,1930740,2803778,1930740,0,1930740,1930740,0
2002-04-07,0,2306761,38567,0,0,0,2306761,2361962,0,310899
1990-03-25,0,439257,576116,0,316402,0,1213501,533832,0,1049524


In [15]:
(
    parties_for_elections.pipe(
        lambda df: (df / total_votes_by_election[df.columns])
    ).loc[lambda df: (df > 0.03).sum(axis=1) > 2]
    .rename(columns=relevant_elections.set_index("election_id")["start_date"].to_dict())
    .T.style.background_gradient(
        axis=None,
    )
    .set_caption('Percentage of votes gained by political parties in Hungarian elections')
)

party,DK,FIDESZ,FKGP,JOBBIK,KDNP,LMP,MDF,MSZP,PM,SZDSZ
election_id,,,,,,,,,,
2018-04-08,0.056236,0.477945,0.000000,0.199823,0.477945,0.073217,0.000000,0.124747,0.124747,0.000000
2010-04-11,0.000000,0.527795,0.000000,0.166798,0.527795,0.074729,0.026398,0.193125,0.000000,0.000000
2006-04-09,0.000000,0.421047,0.000152,0.021574,0.421047,0.000000,0.050364,0.432851,0.000000,0.064871
1998-05-10,0.000000,0.282987,0.138349,0.000000,0.025468,0.000000,0.030500,0.323814,0.000000,0.078584
2014-04-06,0.263551,0.438529,0.001629,0.207905,0.438529,0.054295,0.000000,0.263551,0.263551,0.000000
1994-05-08,0.000000,0.070447,0.088537,0.000000,0.070446,0.000000,0.117695,0.331218,0.000000,0.198041
2022-04-03,0.361846,0.525465,0.000000,0.361846,0.525465,0.361846,0.000000,0.361846,0.361846,0.000000
2002-04-07,0.000000,0.411576,0.006881,0.000000,0.000000,0.000000,0.411576,0.421425,0.000000,0.055471
1990-03-25,0.000000,0.089824,0.117810,0.000000,0.064701,0.000000,0.248149,0.109164,0.000000,0.214618
